In [34]:
import numpy as np
import pandas as pd
from characters.sparkle import Sparkle
from characters.qingque import Qingque
from characters.silver_wolf import Silver_Wolf
from characters.fu_xuan import Fu_Xuan
from utils.arena import Arena
from utils.data_table import Action_Table, Qingque_Probability_Table

In [35]:
qingque_speed = 98

In [36]:
silver_wolf_speed = 161 

In [37]:
sparkle_speed = 160.1

In [38]:
fu_xuan_speed = 134
fu_xuan_technique = True

In [39]:
def Initialize_QQ_Mono_Quantum():
  arena = Arena()
  QQ = Qingque(arena = arena, Spd = qingque_speed)
  SW = Silver_Wolf(arena = arena, Spd = silver_wolf_speed)
  FX = Fu_Xuan(arena = arena, tech = fu_xuan_technique, Spd = fu_xuan_speed, ER = 1.194)
  Spk = Sparkle(arena = arena, recipient = QQ, Spd = sparkle_speed, ER = 1.194, tech = True)
  return arena, QQ, Spk, SW, FX

In [40]:
action_dataframe = Action_Table()
probability_dataframe = Qingque_Probability_Table()

In [41]:
def QQ_Mono_Quantum(cycles = 20, verbose = False, action_df = action_dataframe, prob_df = probability_dataframe):
    arena, qq, spk, sw, fx = Initialize_QQ_Mono_Quantum()
    if spk.tech:
        arena.sp += 3
        spk.tech = False
    while arena.cycle < cycles:
        characters = [qq, spk, sw, fx]
        current_gauges = []
        
        #Make a list of the current action gauges
        for chara in characters:
            current_gauges.append(chara.action_gauge)
        if verbose:
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")
            
        #Find the lowest gauge
        lowest_gauge = min(current_gauges)
        
        #Cycle through all characters and advance them forward by the lowest gauge. Should lead to 1 character with AG of 0
        for chara in characters:
            chara.action_advance(lowest_gauge)
        arena.action_advance(lowest_gauge)
        
        #Qingque's Passive Tile Draw
        qq.tiles += 1 
        if arena.sp >7:
            arena.sp = 7
        #Determine the next character to act based on who has 0 action gauge
        if fx.action_gauge == 0:
        
        #Fu Xuan's actions
            if verbose:
                print("Fu Xuan takes a turn")
            if arena.sp > 0 and fx.skill_counter < 0:
                action_df.add_row(arena, fx, "Skill", -1)
                ##TODO: Add FX's 12% CR Buff:
                fx.skill(verbose = verbose)
  
            else:
                action_df.add_row(arena, fx, "Basic", 1)
                fx.basic(verbose = verbose)

            fx.update_buffs()
            fx.action_reset()

        elif sw.action_gauge == 0:
        
        #Silver Wolf's actions
            if verbose:
                print("Silver Wolf takes a turn")
            if arena.sp > 0 and sw.skill_counter <= 0:
                action_df.add_row(arena, sw, "Skill", -1)
                sw.skill(verbose = verbose)

            else:
                action_df.add_row(arena, sw, "Basic", 1)
                sw.basic(verbose = verbose)

            sw.update_buffs()
            sw.action_reset()

        elif spk.action_gauge ==0:
        
        #Sparkle's actions
            if verbose:
                print("Sparkle takes a turn:")
            if arena.sp > 0:
                action_df.add_row(arena, spk, "Skill", -1)
                spk.skill(verbose = verbose)
            else:
                action_df.add_row(arena, spk, "Basic", 1)
                spk.basic(verbose = verbose)
            if spk.energy >= spk.max_energy:
                action_df.add_row(arena, spk, "Ultimate", 4)
                spk.ult(verbose = verbose)
                
            
            spk.update_buffs()
            spk.action_reset()

        elif qq.action_gauge == 0:
            if qq.tile_battle == True:
                arena.sp += 1
                qq.tile_battle = False
            passive_tiles = qq.tiles
            qq.check_hh(threshold=0.7)

            if qq.hidden_hand == True:
                sp_consumed = int(np.round(qq.likely_multiplier))
                if qq.eidolon >= 6:
                    sp_consumed -= 1
                action_df.add_row(arena, qq, "Enhanced Basic", sp_consumed)
                qq.tiles = 0
            else: 
                sp_consumed = int(arena.sp - 1)
                action_df.add_row(arena, qq, "Enhanced Basic", sp_consumed)
            prob_df.add_row(arena, qq, passive_tiles)
            arena.sp -= sp_consumed
            #qq.update_buffs(verbose)
            qq.action_reset()
        
        else:
            print(f"Action Gauge error!")
            print(f"Current AG: {arena.action_gauge}, current SP: {arena.sp}, current cycle: {arena.cycle}")
            print(f"QQ AG: {current_gauges[0]}, Spk AG: {current_gauges[1]}, SW AG: {current_gauges[2]}, FX AG: {current_gauges[3]}")

        del current_gauges




In [42]:
QQ_Mono_Quantum()

c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)
c:\Users\rainf\OneDrive\Documents\Git Projects\HSR_Action_Simulator-1\utils\data_table.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index = True)


In [43]:
pd.set_option("display.max_rows", None)
#action_dataframe.df

In [44]:
probability_dataframe_styled = probability_dataframe.df.style.set_caption(f"Qingque speed = {qingque_speed}, Sparkle speed = {sparkle_speed}, Silver Wolf Speed = {silver_wolf_speed}, Fu Xuan speed = {fu_xuan_speed}")
probability_dataframe_styled

,Cycle,Action Gauge,Current SP,Current Tiles,P(Success),Average # Skills,Most Likely # Skills,SP Consumed
0,0.000000,62.560962,5,3,0.967908,2.124346,2.284459,2.000000
1,0.000000,125.021924,7,4,0.995798,1.898525,2.239724,2.000000
2,1.000000,187.482886,6,4,0.990547,1.861762,2.226879,2.000000
3,1.000000,249.943848,2,4,0.762222,1.028889,1.666667,2.000000
4,2.000000,312.404809,6,4,0.990547,1.861762,2.226879,2.000000
5,3.000000,374.865771,6,4,0.990547,1.861762,2.226879,2.000000
6,3.000000,437.326733,3,3,0.838962,1.568573,2.053498,2.000000
7,4.000000,499.787695,7,4,0.995798,1.898525,2.239724,2.000000
8,5.000000,562.248657,5,4,0.978734,1.790886,2.197988,2.000000
9,5.000000,624.709619,3,4,0.892812,1.420658,1.987654,2.000000
